In [10]:
import argparse
import logging
import signal
import socket
import sys
import time
import random
from pathlib import Path
import string
#from imutils.video import VideoStream
import cv2
import easygopigo3
from picamera import PiCamera
import imutils as imutils
from joblib import load
import atexit
#import skimage.measure
import numpy as np


In [12]:
# Create instance of the GoPiGo class
gpg = easygopigo3.EasyGoPiGo3()
c = PiCamera()
c.capture('./picture.jpg')
#c.resolution =(640,480)
#c.capture(image,'rgb')
#dataset_path = "/home/pi/group-f/test-sk-learn/TestData"


PiCameraMMALError: Failed to enable connection: Out of resources

In [6]:
# Initialize the button to start/stop the car
# This is using 'Analog/Digital2' on the GoPiGo board.
go_button = gpg.init_button_sensor("AD2")
RELEASED = 0
PRESSED = 1
button_state = RELEASED

In [7]:
# Initialize the state of the car in the stopped state
CAR_GO = 1
CAR_STOP = 0
go_state = CAR_STOP
tempvar = 10

In [ ]:
# Set the speed.  This is a value that can be experimented with.  You have to
# maintain the right balance between speed, turning rate and inference speed.
WHEEL_SPEED_CONSTANT = 30

# turn rate.These values determine how aggresive the turn will be.
# it does this by changing the power to the wheels
# left_turn = (leftwheelspeedfactor, rightwheelspeedfactor)
# for a left turn we reduce the speed of the left wheel and increase the speed of the right wheel.  This will
# cause the car to turn or rotate to the left.  These are values that have to be experimented with.
# (left multiplier, right multiplier)
left_turn = (0.4, 1.0)
right_turn = (1.0, 0.4)

In [ ]:
# Friendly names for the direction predictions
directions = ["Left", "Straight", "Right", "wrong" ]


In [ ]:
def signal_handler(sig, frame):
    """
    Exit gracefully
    :param sig:
    :type sig:
    :param frame:
    :type frame:
    :return:
    :rtype:
    """
    print("You pressed ctrl-c resetting GPG and exiting")
    gpg.reset_all()
    sys.exit()

In [ ]:
def save_image(image_to_save):
    """
    Sometimes it is helpful to save the images the car is seeing while it is driving to help collect
    training images or to debug the model.
    :param image_to_save:
    :type image_to_save:
    :return:
    :rtype:
    """
    global saved_image_index
    print(f"Write File: ./route_images/image_{saved_image_index}.jpg")
    cv2.imwrite(f"./route_images/image_{saved_image_index}.jpg", image_to_save)
    saved_image_index += 1

In [ ]:
def load_model():
    model = load('/home/pi/group-f/test-sk-learn/logRec.sav')
    return model


In [ ]:
if __name__ == '__main__':

    time.sleep(2)
    #signal.signal(signal.SIGINT, signal_handler)
    print('test')
    model = load_model()
    print('test0')
    loop_count = 0
    
    
    time.sleep(3) # just give the system time to settle out

    while True:
        
    
        s = time.time()
        

        image =np.empty((480,640,3),dtype=np.uint8)
        camera.capture(image,'rgb')
        
        #c.capture('/tmp/picture.jpg')
        #img = cv2.imread('/tmp/picture.jpg')
        

        print('test1')

        if img is not None: 
            print('test2')
            # get a cropped, black and white image and the rgb roi
            #dim = (int(320),int(240))
            #dim = (H*0.5,W*0.5)
            #img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            
            (H, W, C) = img.shape
            #roi = img[int(H*0.75):int(H), int(W*0.2):int(W*0.8)]
            
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            (T, thresh) = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
            
            image = skimage.measure.block_reduce(thresh, (6,6), np.max)
            # check to see if we need to save the raw image

            # flatten the image so we can run it through the ML model
            flatten_image = image.flatten()

            # from the flattened image, predict which direction to go
            sm = time.time()
            prediction = model.predict([flatten_image])
            em = time.time()
            # print(f"Predict time: {(em-sm)} seconds")

            direction = prediction[0]
            # print(f"Predicted direction: {directions[direction]}")
            p = Path(f"{dataset_path}/{direction}")
            p.mkdir(exist_ok=True, parents=True)
            filename = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
            full_filename = f"{dataset_path}/{direction}/{filename}_{direction}.jpg"
            cv2.imwrite(full_filename, thresh)
            full_filename = f"{dataset_path}/{direction}/{filename + 'vanlig'}_{direction}.jpg"
            cv2.imwrite(full_filename, img)
            
    

            # based on the prediction, change the wheel power to make turns
            if direction == 3:
                direction = tempvar

            if direction == 0: # left
                gpg.set_motor_power(gpg.MOTOR_LEFT, WHEEL_SPEED_CONSTANT * left_turn[0])
                gpg.set_motor_power(gpg.MOTOR_RIGHT, WHEEL_SPEED_CONSTANT * left_turn[1])
                tempvar = direction
            elif direction == 1: #straight
                gpg.set_motor_power(gpg.MOTOR_LEFT, WHEEL_SPEED_CONSTANT)
                gpg.set_motor_power(gpg.MOTOR_RIGHT, WHEEL_SPEED_CONSTANT)
                tempvar = direction
            elif direction == 2: # right
                gpg.set_motor_power(gpg.MOTOR_LEFT, WHEEL_SPEED_CONSTANT * right_turn[0])
                gpg.set_motor_power(gpg.MOTOR_RIGHT, WHEEL_SPEED_CONSTANT * right_turn[1])
                tempvar = direction
            

            else:
                print(f"Unknown direction: {direction}")

        e = time.time()
        # print(f"Loop Time: {(e-s)} seconds")

    gpg.reset_all()
    gpg.stop()
    
def exit_handler():
    gpg.stop()
    print("Stopped and exit")
atexit.register(exit_handler)
